In [1]:
rm(list=ls())
suppressPackageStartupMessages(require(skimr))
suppressPackageStartupMessages(require(readxl))
suppressPackageStartupMessages(require(stringr))
suppressPackageStartupMessages(require(dplyr))
suppressPackageStartupMessages(require(ggplot2))
suppressPackageStartupMessages(require(forecast))

In [2]:
#------------------------------------------------------
# data pre treatment (similar to univariate case)

df <- read_excel("data/data.xlsx",sheet = 3)
tmp <-df%>% filter(aire=="44_GRAND EST" & champ=="ESS")

df <-tmp[-which(!is.na(tmp$secret_stat)),]%>%select(-secret_stat)
df <- df %>% select(-champ,-type_aire,-type_naf,-aire)
df <- df[,c(TRUE,str_detect(colnames(df),"nb_etab")[-1])]
tmp <- t(df)
colnames(tmp) <- tmp[1,]
tmp <-tmp[-1,]
df <- tmp
tmp <-df%>%as.data.frame
tmp <-cbind("date"=rep(NA,nrow(tmp)),tmp)
for (i in 1:nrow(tmp)){
    tmp.format.date <- strsplit(rownames(df)[i],"nb_etab")[[1]][2]
    subsplit <-strsplit(tmp.format.date,"T")[[1]]
    year<-subsplit[1]
    trimester <- subsplit[2]
    month <-switch(trimester,
        "1"="01",
        "2"="04",
        "3"="07",
        "4"="10"
    )
    tmp[i,1] <- paste(year,month,"01",sep="-")
}
tmp$date <-as.Date(tmp$date)
rownames(tmp) <- NULL

df.indexes <-which(tmp$date < as.Date("2020-01-01"))
train.indexes<- which(tmp$date < as.Date("2018-01-01") )
test.indexes <-which(tmp$date>= as.Date("2018-01-01") & tmp$date <as.Date("2020-01-01") )
tmp <-tmp %>%select(-date)%>% mutate_all(as.numeric)
#----------------------------
# Eliminate total serie
tmp <- tmp[,-ncol(tmp)]
df.ts <- ts(tmp[df.indexes,],start=c(2010,1),frequency=4)# S'arrete à 2019 Q4
train.ts <-ts(tmp[train.indexes,],start=c(2010,1),frequency = 4)# S'arrete à 2017 Q4
test.ts <-ts(tmp[test.indexes,],start = c(2018,1),frequency = 4)# Commence a 2018 Q1 --> 2019 Q4
global.ts <- ts(tmp,start=c(2010,1),frequency=4)# Commence a 2010 Q1 --> 2021 Q2
covid.ts <- ts(tmp[(df.indexes[length(df.indexes)]+1):nrow(tmp),],start=c(2020,1),frequency = 4)# Commence a 2020 Q1 --> 2021 Q2

In [3]:
require(tensorflow)
require(keras)

Loading required package: tensorflow

Loading required package: keras



In [7]:
lab.to.num <-cbind("lab"=colnames(df.ts),"num"=1:length(colnames(df.ts))) %>%as.data.frame
lab.to.num$num <- as.numeric(lab.to.num$num)
colnames(df.ts) <-lab.to.num$num

In [12]:
df <- df.ts %>% as.data.frame()
df.scaled <-scale(df)
centered_scale <-attr(df.scaled,"scaled:center")
sd_scale <-attr(df.scaled,"scaled:scale")

In [15]:
#----------------
# We d like to do a 6 Quarter forecast,
# We need to base each prediction on 6 points
prediction <- 6
lag <- prediction

In [17]:
df.scaled <- as.matrix(df.scaled)
df.scaled

1,2,3,4,5,6,7,8,9,10,⋯,17,18,19,20,21,22,23,24,25,26
1.258286041,-2.1070676,0.57533001,2.08893236,0.83297113,-1.70108329,-2.06805496,-1.6105164,-1.60977544,2.67530080,⋯,1.153590558,-0.59215079,0.6104969,0.5768366,2.20804706,0.36963901,0.83258771,-1.75151624,0.16352704,-1.38275175
1.174678995,-2.1722515,1.67119669,2.08893236,1.03476764,-2.57906177,-1.98483343,-1.4204850,-1.64840232,2.24205371,⋯,1.245236282,0.15582916,1.0988944,-0.1223593,2.38540024,0.81768629,0.96833571,-0.19714036,0.90381838,-1.18089018
1.105006457,-2.0418838,1.67119669,2.08893236,0.02578511,-2.57906177,-0.98617510,-1.0404221,-1.45526790,2.53088511,⋯,2.894859324,-0.59215079,1.0988944,0.5768366,1.14392799,1.26573356,1.19458237,-1.06910731,0.51709903,-0.37344390
1.091071949,-2.6285383,1.67119669,2.08893236,0.87781480,-1.70108329,-1.73516885,-1.2304536,-1.33938724,2.67530080,⋯,0.741184797,0.15582916,1.0988944,0.5768366,0.07980893,1.26573356,1.14933303,-0.04549393,0.67178677,-0.97902861
1.063202934,0.3699183,1.12326335,1.83024105,0.49664362,0.49386289,-0.81973205,-1.4204850,-1.45526790,1.51997522,⋯,0.832830522,-0.09349749,1.5872918,-0.4719573,-0.09754425,1.26573356,0.96833571,-0.84163767,-0.17899582,-1.38275175
1.035333919,0.4351021,1.12326335,1.31285842,0.74328379,0.05487365,-0.73651052,-1.2304536,-1.22350659,0.94231243,⋯,0.878653384,-0.34282414,0.1220994,-0.4719573,0.96657482,0.81768629,1.19458237,0.40944535,0.38450953,-1.78647489
0.965661381,0.3047345,1.67119669,2.34762368,-0.53476074,0.05487365,-0.81973205,-1.6105164,-1.14625282,0.79789673,⋯,2.482453563,-1.34013074,0.6104969,0.5768366,0.61186846,0.81768629,1.37557969,-0.99328410,0.24087091,-1.18089018
0.937792365,0.3047345,0.57533001,0.53678448,0.36211262,-0.82310482,-0.81973205,-1.4204850,-1.03037216,0.22023394,⋯,0.466247624,0.15582916,-0.3662981,0.5768366,0.61186846,1.71378084,0.92308637,-0.61416803,0.24087091,-0.77716704
0.826316304,0.4351021,-0.52053667,-0.23928947,0.51906546,-0.82310482,-0.65328899,-1.2304536,-1.14625282,-0.79067595,⋯,0.282956175,-0.84147744,-0.8546956,0.5768366,0.07980893,2.16182812,0.69683971,-1.10701892,0.27401829,-0.57530547
0.798447289,0.3699183,-0.52053667,-0.23928947,0.60875279,-0.38411558,-0.48684594,-1.0404221,-0.79861085,-0.50184455,⋯,0.282956175,-0.84147744,0.1220994,0.5768366,0.07980893,1.48975720,0.65159038,0.44735696,0.84857276,-0.57530547


In [20]:
df.1.scaled <-df.scaled[,1]

In [22]:
1:length(df.1.scaled)-lag -prediction +1

[1] -10  -9  -8  -7  -6  -5  -4  -3  -2  -1   0   1   2   3   4   5   6   7   8
[20]   9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27
[39]  28  29

In [26]:
df.1.train <-sapply(1:(length(df.1.scaled)- lag -prediction +1),
function(x){df.1.scaled[x:(x+lag-1)]}
)
df.1.train <- as.matrix(df.1.train)
df.1.train <-t(df.1.train)

In [28]:
df.scaled[1,]

1          2          3          4          5          6          7 
 1.2582860 -2.1070676  0.5753300  2.0889324  0.8329711 -1.7010833 -2.0680550 
         8          9         10         11         12         13         14 
-1.6105164 -1.6097754  2.6753008 -1.3563683  0.1219673  0.9845100  2.6943198 
        15         16         17         18         19         20         21 
 1.3649763  1.9487491  1.1535906 -0.5921508  0.6104969  0.5768366  2.2080471 
        22         23         24         25         26 
 0.3696390  0.8325877 -1.7515162  0.1635270 -1.3827518

In [30]:
df.train.arr <- array(
    data=as.numeric(unlist(df.1.train)),
    dim=c(nrow(df.1.train),lag,1)
)

In [34]:
# Now a 3D form
df.train.arr 

, , 1

              [,1]         [,2]         [,3]         [,4]         [,5]
 [1,]  1.258286041  1.174678995  1.105006457  1.091071949  1.063202934
 [2,]  1.174678995  1.105006457  1.091071949  1.063202934  1.035333919
 [3,]  1.105006457  1.091071949  1.063202934  1.035333919  0.965661381
 [4,]  1.091071949  1.063202934  1.035333919  0.965661381  0.937792365
 [5,]  1.063202934  1.035333919  0.965661381  0.937792365  0.826316304
 [6,]  1.035333919  0.965661381  0.937792365  0.826316304  0.798447289
 [7,]  0.965661381  0.937792365  0.826316304  0.798447289  0.784512781
 [8,]  0.937792365  0.826316304  0.798447289  0.784512781  0.784512781
 [9,]  0.826316304  0.798447289  0.784512781  0.784512781  0.659102212
[10,]  0.798447289  0.784512781  0.784512781  0.659102212  0.617298689
[11,]  0.784512781  0.784512781  0.659102212  0.617298689  0.561560659
[12,]  0.784512781  0.659102212  0.617298689  0.561560659  0.464019105
[13,]  0.659102212  0.617298689  0.561560659  0.464019105  0.338608536
[14,]  0.617298689  0.561560659  0.464019105  0.338608536  0.338608536
[15,]  0.561560659  0.464019105  0.338608536  0.338608536  0.296805013
[16,]  0.464019105  0.338608536  0.338608536  0.296805013  1.648452256
[17,]  0.338608536  0.338608536  0.296805013  1.648452256  0.115656414
[18,]  0.338608536  0.296805013  1.648452256  0.115656414 -0.037623171
[19,]  0.296805013  1.648452256  0.115656414 -0.037623171 -0.009754155
[20,]  1.648452256  0.115656414 -0.037623171 -0.009754155 -0.037623171
[21,]  0.115656414 -0.037623171 -0.009754155 -0.037623171 -0.121230217
[22,] -0.037623171 -0.009754155 -0.037623171 -0.121230217 -0.302378816
[23,] -0.009754155 -0.037623171 -0.121230217 -0.302378816 -0.358116847
[24,] -0.037623171 -0.121230217 -0.302378816 -0.358116847 -0.469592908
[25,] -0.121230217 -0.302378816 -0.358116847 -0.469592908 -0.553199954
[26,] -0.302378816 -0.358116847 -0.469592908 -0.553199954 -0.678610523
[27,] -0.358116847 -0.469592908 -0.553199954 -0.678610523 -0.887628138
[28,] -0.469592908 -0.553199954 -0.678610523 -0.887628138 -1.068776737
[29,] -0.553199954 -0.678610523 -0.887628138 -1.068776737 -1.194187306
              [,6]
 [1,]  1.035333919
 [2,]  0.965661381
 [3,]  0.937792365
 [4,]  0.826316304
 [5,]  0.798447289
 [6,]  0.784512781
 [7,]  0.784512781
 [8,]  0.659102212
 [9,]  0.617298689
[10,]  0.561560659
[11,]  0.464019105
[12,]  0.338608536
[13,]  0.338608536
[14,]  0.296805013
[15,]  1.648452256
[16,]  0.115656414
[17,] -0.037623171
[18,] -0.009754155
[19,] -0.037623171
[20,] -0.121230217
[21,] -0.302378816
[22,] -0.358116847
[23,] -0.469592908
[24,] -0.553199954
[25,] -0.678610523
[26,] -0.887628138
[27,] -1.068776737
[28,] -1.194187306
[29,] -1.235990829

In [39]:
dim(df.train.arr)
df.train.arr %>%head()

[1] 29  6  1

, , 1

         [,1]      [,2]      [,3]      [,4]      [,5]      [,6]
[1,] 1.258286 1.1746790 1.1050065 1.0910719 1.0632029 1.0353339
[2,] 1.174679 1.1050065 1.0910719 1.0632029 1.0353339 0.9656614
[3,] 1.105006 1.0910719 1.0632029 1.0353339 0.9656614 0.9377924
[4,] 1.091072 1.0632029 1.0353339 0.9656614 0.9377924 0.8263163
[5,] 1.063203 1.0353339 0.9656614 0.9377924 0.8263163 0.7984473
[6,] 1.035334 0.9656614 0.9377924 0.8263163 0.7984473 0.7845128

Every column is a lagged version of the previous one. The lag one is lagged by 5 steps comparing to the first one

In [47]:
y.train.data <-t(sapply(
    (1+lag):(length(df.scaled[,1])- prediction +1),
    function(x){df.1.train[x:(x+prediction-1)]}
))
y_train_arr <- array(
    data = as.numeric(unlist(y.train.data)),
    dim = c(
        nrow(y.train.data),
        prediction,
        1
    )
)

In [48]:
y_train_arr %>% dim

[1] 29  6  1

In [49]:
LSTM_model <- keras_model_sequential()

LSTM_model %>% layer_lstm(
    units=50,# Size of the layer
    batch_input_shape=c(1,6,1),# Batch size ,timesteps,features
    return_sequences = T,
    stateful = TRUE
)%>%
layer_dropout(rate=0.5)%>%
layer_lstm(units=50,
return_sequences = T,
stateful=T)%>%
layer_dropout(rate=0.5)%>%
time_distributed(keras::layer_dense(units=1))

In [50]:
LSTM_model%>% compile(loss='mae',optimizer='adam',metrics='accuracy')

summary(LSTM_model)

Model: "sequential_2"
________________________________________________________________________________
 Layer (type)                       Output Shape                    Param #     
 lstm_3 (LSTM)                      (1, 6, 50)                      10400       
 dropout_3 (Dropout)                (1, 6, 50)                      0           
 lstm_2 (LSTM)                      (1, 6, 50)                      20200       
 dropout_2 (Dropout)                (1, 6, 50)                      0           
 time_distributed_1 (TimeDistribute  (1, 6, 1)                      51          
 d)                                                                             
Total params: 30,651
Trainable params: 30,651
Non-trainable params: 0
________________________________________________________________________________


In [51]:
LSTM_model %>% fit(
    x=df.train.arr,
    y=y_train_arr,
    batch_size=1,
    epochs=20,
    verbose=0,
    shuffle=FALSE
)